# Torch example

Apply example from [this source](https://torcheeg.readthedocs.io/en/latest/introduction_by_example.html) on our data set

In [17]:
from pathlib import Path
from utils.dataSet import build_NumpyDataset

from torcheeg.model_selection import KFold

import torch 
import torch.nn as nn
from torch.utils.data import DataLoader

In [18]:
eeg_dir  = Path('/home/admin/work/NetworkMachineLearning_2023/EEGDataset')
subjects = ['sub-01', 'sub-02', 'sub-03']
task     = 'faces'

# Load data
dataset = build_NumpyDataset(eeg_dir, subjects, task)

The target folder already exists, if you need to regenerate the database IO, please delete the path ./io/numpy.


In [19]:
# Define data splitting method
k_fold = KFold(n_splits=10, split_path='./io/split', shuffle=True)

In [29]:
# Define model 
class CNN(torch.nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.Conv1 = nn.Sequential(
            # temporal convolution
            nn.Conv2d(1, 64, kernel_size=(1,11), stride=1),
            nn.ReLU(),

            # spatial convolution
            nn.Conv2d(64, 64, kernel_size=(3,1), stride=1),
            nn.ReLU(),

            # pooling
            nn.MaxPool2d((2,3), stride=1)
        )
        self.Conv2 = nn.Sequential(
            # temporal convolution
            nn.Conv2d(64, 128, kernel_size=(1,10), stride=1),
            nn.ReLU(),

            # spatial convolution
            nn.Conv2d(128, 128, kernel_size=(2,1), stride=1),
            nn.ReLU(),

            # pooling
            nn.MaxPool2d((2,3), stride=1)
        )
        self.Conv3 = nn.Sequential(
            # temporal convolution
            nn.Conv2d(128, 256, kernel_size=(1,3), stride=1),
            nn.ReLU(),

            # spatial convolution
            nn.Conv2d(256, 256, kernel_size=(2,1), stride=1),
            nn.ReLU(),

            # pooling
            nn.MaxPool2d((2,2), stride=1)
        )

        self.lin1 = nn.Linear(15 * 15 * 256, 1024)
        self.lin2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = x[:,None,:,:]  # reshape with 1 channel

        x = self.Conv1(x)
        x = self.Conv2(x)
        x = self.Conv3(x)
        print(x.shape)
        
        x = x.flatten(start_dim=1)
        x = self.lin1(x)
        x = self.lin2(x)
        return x

In [30]:
device = 'cpu' if torch.cuda.is_available() else 'cpu' 
loss_fn = nn.CrossEntropyLoss()
batch_size = 32 

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_idx, batch in enumerate(dataloader):
        X = batch[0].to(device)
        y = batch[1]['trial'].to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            loss, current = loss.item(), batch_idx * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def valid(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad():
        for batch in dataloader:
            X = batch[0].to(device)
            y = batch[1]['trial'].to(device)

            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {val_loss:>8f} \n")

In [31]:
for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):

    model = CNN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    epochs = 5
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_loader, model, loss_fn, optimizer)
        valid(val_loader, model, loss_fn)
    print("Done!")

Epoch 1
-------------------------------
torch.Size([32, 256, 121, 599])


In [12]:
import numpy as np

arr = np.array(['FACES', 'SCRAMBLED', 'FACES'])

labels = np.char.lower(np.unique(arr))
print(np.char.upper(labels[0]))

FACES


In [9]:
type(labels[0])

numpy.str_